In [1]:
import cPickle as pickle
import pandas as pd
from collections import defaultdict, Counter
import numpy as np
from tqdm import tqdm

In [17]:
data=pickle.load(open("results_.pickle", 'rb'))

In [18]:
data.head(5)

,corpus_id,docid,ngram_id,count,doc_length,tfidf,tf,tfidf_var,tfidf_fullvar
0,0,137827,23737,107,11872,0.050491,0.009013,0.039271,1.172770e-08
1,0,137827,62,89,11872,0.014588,0.007497,0.012235,7.028444e-07
2,0,137827,488,67,11872,0.016907,0.005644,0.014718,9.472815e-10
3,0,137827,22152,53,11872,0.021899,0.004464,0.017032,2.374237e-11
4,0,137827,137671,39,11872,0.018403,0.003285,0.014314,5.678782e-10


In [4]:
corpus_list=range(3)

In [5]:
corpus_docid_dict=defaultdict(list)
corpus_num_doc=defaultdict(int)
for corpus in corpus_list:
    corpus_docid_dict[corpus]=data[data['corpus_id']==corpus]['docid'].drop_duplicates().tolist()
    corpus_num_doc[corpus]=len(corpus_docid_dict[corpus])
    
corpus_ngram_dict=defaultdict(list)
for corpus in corpus_list:
    corpus_ngram_dict[corpus]=data[data['corpus_id']==corpus]['ngram_id'].drop_duplicates().tolist()
    
corpus_docid_ngram_dict=defaultdict()
for corpus in corpus_list:
    corpus_docid_ngram_dict[corpus]=defaultdict(list)
    for docid in corpus_docid_dict[corpus]:
        corpus_docid_ngram_dict[corpus][docid]=data[(data['corpus_id']==corpus)&(data['docid']==docid)]['ngram_id'].drop_duplicates().tolist()
        

In [6]:
ngram_list=data['ngram_id'].drop_duplicates().tolist()

In [13]:
HHI_intra=defaultdict()
for corpus in corpus_list:
    HHI_intra[corpus]=defaultdict(float)
    for ngram in tqdm(corpus_ngram_dict[corpus]):
         HHI_intra[corpus][ngram]=reduce(lambda x,y: x*y, 
                                         map(lambda x: 100*x*100*x, 
                                                               data[(data['corpus_id']==corpus)&
                                                                    (data['ngram_id']==ngram)]['tf'].tolist()))/10000

100%|█████████████████████████████████████| 8689/8689 [00:13<00:00, 645.02it/s]


In [14]:
sum_tf_corpus_ngram=defaultdict()
for corpus in corpus_list:
    for ngram in tqdm(corpus_ngram_dict[corpus]):
        sum_tf_corpus_ngram[ngram]=defaultdict(float)
        sum_tf_corpus_ngram[ngram][corpus]=sum(data[(data['corpus_id']==corpus)&(data['ngram_id']==ngram)]['tf'].tolist())

100%|█████████████████████████████████████| 8689/8689 [00:13<00:00, 647.03it/s]


In [15]:
HHI_inter=defaultdict()
for corpus in corpus_list:
    HHI_inter[corpus]=defaultdict(float)
    for ngram in corpus_ngram_dict[corpus]:
        HHI_inter[corpus][ngram]= pow(sum_tf_corpus_ngram[ngram][corpus]*100/sum(sum_tf_corpus_ngram[ngram].values()),2)/10000

In [20]:
indexes=[]
tfidf_HHI=[]
for index,row in data.iterrows():
    indexes.append(index)
    tfidf_HHI.append(row['tfidf']*HHI_inter[row['corpus_id']][row['ngram_id']]*(1-HHI_intra[row['corpus_id']][row['ngram_id']]))

data=pd.concat([data, pd.Series(data=tfidf_HHI, index=indexes, name='tfidf_HHI')],axis=1)

    

    

In [21]:
data.head(5)   

,corpus_id,docid,ngram_id,count,doc_length,tfidf,tf,tfidf_var,tfidf_fullvar,tfidf_HHI
0,0,137827,23737,107,11872,0.050491,0.009013,0.039271,1.172770e-08,0.050491
1,0,137827,62,89,11872,0.014588,0.007497,0.012235,7.028444e-07,0.000000
2,0,137827,488,67,11872,0.016907,0.005644,0.014718,9.472815e-10,0.000000
3,0,137827,22152,53,11872,0.021899,0.004464,0.017032,2.374237e-11,0.021899
4,0,137827,137671,39,11872,0.018403,0.003285,0.014314,5.678782e-10,0.018403


In [22]:
pickle.dump(data, open("results_.pickle", 'wb'))